**ACP-SRC-CKSAAP**

In [1]:
!pip install wget
import os
from google.colab import drive
drive_path = '/content/drive/MyDrive/ACP_SRC_Ehtisham_Share/'

os.chdir(drive_path)
os.getcwd() 

from utilities.utils import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
File already exist.


In [2]:
experiment = 'ACP740_LinearPCA_BASIS_PURSUIT'
gaps = [8] # defined (K) in composition of K-spaced Amino-Acid-Pairs (CKSAAP)
num_folds = 5 # number of K-Folds (cross validation folds)
normalization = True  # {‘l1’, ‘l2’, ‘max’}, default=’l2’ 
oversampling = True   # default method --> KMeansSMOTE
train_stats = False   # generate training statistics

pc_list=[10,20,30,40,50,60,70,80,90,100,110,150,175,200,225,250,300,350,400,450,500,550,600]

exp_path  = drive_path + experiment
os.makedirs(exp_path, exist_ok = True)
os.chdir(exp_path)
os.getcwd() 

'/content/drive/.shortcut-targets-by-id/1QJSKQuA8-V4nGj95_-knAqK8JPdNQ_vl/ACP_SRC_Ehtisham_Share/ACP740_LinearPCA_BASIS_PURSUIT'

In [ ]:
if ('ACP740' in experiment):
  dataset = 'ACP740'
  [DataX, LabelY] = Convert_Seq2CKSAAP(prepare_feature_acp740(), gap=gaps[0])

if ('LinearPCA' in experiment):
  dimension_reduction = 'PCA'

if ('BASIS_PURSUIT' in experiment):
  solver_method = 'BP'   

kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=2)
for num_pc  in pc_list:
  print("For {} number of components(features)".format(num_pc))
  classification_stats = []
  time_stats=[]
  cross_fold_ing=-1  

  for train_index, test_index in kf.split(DataX,np.argmax(LabelY,axis=1)):
      cross_fold_ing = cross_fold_ing + 1
      print('Fold # ', cross_fold_ing)

      # Loading dataset
      print('Loading dataset')
      X_train, X_test = DataX[train_index], DataX[test_index]
      y_train, y_test = LabelY[train_index], LabelY[test_index]
      print('num pos train:', sum(y_train[:,0]==1), 'num neg train:', sum(y_train[:,0]==0))
      y_train = y_train[:,0]
      y_test=y_test[:,0]  
      
      ## pre-processing
      if(normalization):
        normalizer = Normalizer().fit(X_train)  
        X_train = normalizer.transform(X_train)
        X_test = normalizer.transform(X_test)
      
      ## Oversampling
      if(oversampling):
        oversampler = KMeansSMOTE(random_state=42)    
        X_train, y_train = oversampler.fit_resample(X_train, y_train)
        print('After Resampling \n','num pos train:', sum(y_train==1), 'num neg train:', sum(y_train==0))

      ## Dimension reduction
      if(dimension_reduction=='PCA'):
        transformer =transformer = PCA(n_components=num_pc)

      transformer.fit_transform(X_train)
      X_train = transformer.transform(X_train)
      X_test = transformer.transform(X_test)  
      X_train = np.transpose(X_train)
      X_test = np.transpose(X_test)
      y_test_pred,y_test_score,elp_time= Test_SRC(X_train,y_train,X_test,y_test,solver=solver_method,verbose=0, x0=None, ATinvAAT=None, nnz=None, positive=True, tol=1E-4, niter=100, biter=32)

      
      if(train_stats):
        tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi = Calculate_Stats(y_train, y_train_pred)

      t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi = Calculate_Stats(y_test,y_test_pred)
      # print(t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, elp_time)

      ###AUC ROC CURVE
      r_auc = roc_auc_score(y_test,y_test_score)
      r_fpr, r_tpr, _ = roc_curve(y_test,y_test_score)
      # plt.plot(r_fpr, r_tpr, linestyle='--', label='ACP_SRC (AUROC = %0.3f)' % r_auc)
      # del model  # deletes the existing model

      classification_stats.append([t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, r_auc])
      time_stats.append(elp_time)

  Class_Statistics = np.asarray(classification_stats)
  Time_Statistics= np.asarray(time_stats)

  filename = 'ACP_KSRC_STATS_CKSAAP_GAP' + str(gaps[0]) + "_Solver_OP" + '_PC' + str(num_pc) + '.mat'
  savemat(filename,{'Time_Statistics':Time_Statistics,'Class_Statistics':Class_Statistics})
  print('SAVING... '+ +experiment + '/' + filename)
